In [10]:
import pyarrow.parquet as pq
import pyarrow as pa
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
DATA_DIR = '../../data/batches'

In [12]:
!du $DATA_DIR/encAnonymusHTTPRequests-20180217_parsedUAA_parquet.parquet -h

23M	../../data/batches/encAnonymusHTTPRequests-20180217_parsedUAA_parquet.parquet


In [13]:
http = pq.read_table(os.path.join(DATA_DIR, 'encAnonymusHTTPRequests-20180217_parsedUAA_parquet.parquet')).to_pandas()
http = http.sample(frac=1).reset_index()

In [14]:
http.shape

(228428, 35)

In [15]:
http.head()

,index,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,...,LayoutEngineVersionMajor,LayoutEngineNameVersion,LayoutEngineNameVersionMajor,AgentClass,AgentName,AgentVersion,AgentVersionMajor,AgentNameVersion,AgentNameVersionMajor,isBot
0,127901,-3825840030379057683,None,aEnhiIdxNIhT0kYGGn6H1w==,GET,PopLayerPhoto,https://ok.ru/,REQ,1518757128205,None,...,Blink 64.0,Blink 64,Browser,Chrome,64.0.3282.168,64,Chrome 64.0.3282.168,Chrome 64,None,0
1,123737,7693801844659994129,None,BdcBv+ehNVLOgetPIJcI8A==,POST,FriendsPageMRB,https://ok.ru/,REQ,1518808162317,None,...,Blink 63.0,Blink 63,Browser,Opera,50.0.2762.67,50,Opera 50.0.2762.67,Opera 50,None,0
2,184208,6131714660239713839,None,+lBKCQAwVi/6MELjJJq2xA==,POST,videoStatNew,https://ok.ru/videoembed/37705026294,REQ,1518743373837,None,...,Gecko 58.0,Gecko 58,20100101,Browser,Firefox,58.0,58,Firefox 58.0,Firefox 58,0
3,192719,6256074590087211199,None,RYM4/U2orLfldmSqcmXL7Q==,None,CSP,None,CSP,1518733889549,None,...,Blink 64.0,Blink 64,Browser,Opera,51.0.2830.34,51,Opera 51.0.2830.34,Opera 51,None,0
4,124540,-4550623916966314647,None,NavoT98A3kmM90UAEzMGHA==,POST,videoPlayerMetadata,https://www.ok.ru/,REQ,1518779246605,None,...,Blink 63.0,Blink 63,Browser,Opera,50.0.2762.67,50,Opera 50.0.2762.67,Opera 50,None,0


In [5]:
http['target'] = http['isBot']
http.drop('isBot', axis=1, inplace=True)

In [6]:
pq.write_table(pa.Table.from_pandas(http), os.path.join(DATA_DIR, 'http-20180217_1718_parsedUAA_target.parquet'))

In [7]:
http_batches_train = http[http.index < int(http.shape[0]*0.7)]
http_batches_test = http[http.index >= int(http.shape[0]*0.7)]

### Запишем на диск

In [8]:
pq.write_table(pa.Table.from_pandas(http_batches_train), 
               os.path.join(DATA_DIR, 'http-20180217_1718_parsedUAA_batches_train.parquet'))
pq.write_table(pa.Table.from_pandas(http_batches_test), 
               os.path.join(DATA_DIR, 'http-20180217_1718_parsedUAA_batches_test.parquet'))